## Closed-Form Solutions for Robust Acoustic Sensor Localization
##### Bruno Granato
##### Carolina Bez
##### Eduardo Alves
##### Fabiana Fonseca
##### Pedro Lisboa

##### ***Processamento de Sinais de Áudio - 2019.1***
##### ***Escola Politécnica - Universidade Federal do Rio de Janeiro***

---

#### Replicação dos resultados encontrados nas simulações de: 
#### Haddad, Diego & Nunes, Leonardo & Martins, Wallace & Biscainho, Luiz & Lee, Bowon. (2013). Closed-Form Solutions for Robust Acoustic Sensor Localization. 


### Sumário
___

- [Config](#config)
- [Métodos TOF](#tof)
    - [T](#t)
    - [Ta](#ta)
    - [Tab](#tab)
- [Métodos TDOF](#tdof)
- [Simulação I](#sim1)
- [Simulação II](#sim2)
- [Simulação III](#sim3)
- [Simulação IV](#sim4)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

### Config <a name=config></a>

In [3]:
space_dim = 3
n_loudspeakers = 12
n_microphones = 1000
room_dim = np.array([5.2, 7.5, 2.6])
c = 340 # speed of sound

ls_pos = room_dim * np.random.rand(n_loudspeakers, space_dim)
mp_pos = room_dim * np.random.rand(n_microphones, space_dim)

## Métodos TOF <a name="tof"></a>

In [28]:
def solve(H,g):
    H_t = np.transpose(H)
    H_inv = np.linalg.inv(H_t @ H)

    sol = np.dot(H_inv @ H_t, g) 
    return sol

### T solution <a name=t></a>

\begin{equation*}
e_{(T)} =  \begin{bmatrix}
1 & -2s_1^{T} \\
\vdots &\vdots \\
1 & -2s_1^{T}
\end{bmatrix}
\begin{bmatrix}
 ||\textbf{m}||^2 \\
 \textbf{m} 
\end{bmatrix} - 
\begin{bmatrix}
-||s_1||^2 + c^2\hat{t}_1^2 \\
\vdots  \\
-||s_S||^2 + c^2\hat{t}_S^2
\end{bmatrix}
\end{equation*}

In [30]:
def tof_t(t_hat, ls_pos):
    ## generate matrix H
    n_loudspeakers = ls_pos.shape[0]
    H = np.concatenate([np.ones((n_loudspeakers, 1), dtype=np.float64),
                        -2*ls_pos], axis=1)
    g = -np.linalg.norm(ls_pos, ord=2, axis=1)**2 + (c**2)*t_hat**2
    
    return solve(H,g)

### Ta solution <a name=ta></a>

\begin{equation*}
e_{(T)} =  \begin{bmatrix}
1 & -2s_1^{T} & \hat{t}_1^2\\
\vdots &\vdots &\vdots \\
1 & -2s_1^{T} & \hat{t}_S^2
\end{bmatrix}
\begin{bmatrix}
 ||\textbf{m}||^2 \\
 \textbf{m} \\
 a^2
\end{bmatrix} - 
\begin{bmatrix}
-||s_1||^2 \\
\vdots  \\
-||s_S||^2
\end{bmatrix}
\end{equation*}

In [6]:
def tof_ta(t_hat, ls_pos):
    ## generate matrix H
    n_loudspeakers = ls_pos.shape[0]
    t_hat = t_hat.reshape((t_hat.shape[0], 1))
    H = np.concatenate([np.ones((n_loudspeakers, 1), dtype=np.float64),
                        -2*ls_pos, -(c**2)*t_hat**2], axis=1)
    g = -np.linalg.norm(ls_pos, ord=2, axis=1)**2
    return solve(H,g)

### Simulação I <a name=sim1></a>

In [33]:
def run_simulation_one(mp_pos, ls_pos, scale):
    # distance between source and microphone 
    distance = np.apply_along_axis(lambda s: mp_pos - s,axis=1, arr=ls_pos) 
    t_hat_array = np.linalg.norm(distance, ord=2, axis=2) / c
    noise_array = np.random.normal(loc=0, scale=scale, size=(n_loudspeakers,n_microphones))
    t_hat_array += noise_array
    solT = np.apply_along_axis(lambda t_hat: tof_t(t_hat, ls_pos),
                              axis=0, arr=t_hat_array)
    solTa = np.apply_along_axis(lambda t_hat: tof_ta(t_hat, ls_pos),
                              axis=0, arr=t_hat_array)
    return {'T' : solT[0,:], 
            'Ta': solTa[0,:]}

std_range_cm = np.arange(0,6)
std_range = std_range_cm / (c*100)
results = {scale:run_simulation_one(mp_pos, ls_pos, scale) 
                       for scale in std_range}

# Flattening results dictionary for multiindex parsing on pandas
results = {(method, scale) : value   
           for scale, res in results.items()
           for method, value in res.items()}
results
results = pd.DataFrame(results)

In [25]:
m = (np.linalg.norm(mp_pos, ord=2, axis=1)**2)
results.groupby(level=0, axis=1).apply(lambda g: g.sub(m, axis=0).abs()).mean()


T   0.000000    5.770728e-14
    0.000029    5.542726e-02
    0.000059    1.144764e-01
    0.000088    1.779802e-01
    0.000118    2.343198e-01
    0.000147    2.889972e-01
Ta  0.000000    5.967377e-13
    0.000029    8.383153e-02
    0.000059    1.638805e-01
    0.000088    2.430387e-01
    0.000118    3.351062e-01
    0.000147    4.071306e-01
dtype: float64

In [430]:
m = (np.linalg.norm(mp_pos, ord=2, axis=1)**2)
results.sub(m,axis=0).abs().mean()
# ax = (100*results.sub(m,axis=0).abs().mean()).plot(linestyle = '--', marker='o')
# ax.set_ylabel('Mean Error (cm)');
# ax.set_xlabel('Noise Standard Deviation (cm)');

0    1.779299e-14
1    6.311449e-02
2    1.157584e-01
3    1.779312e-01
4    2.546909e-01
5    3.049245e-01
dtype: float64

In [238]:
n_loudspeakers = ls_pos.shape[0]
t_hat = np.linalg.norm((mp_pos[0,:] - ls_pos), ord=2, axis=1) / c
print('M ' + str(mp_pos[0,:]))
# print('lspos ' + str(ls_pos))
simple_tof(t_hat, ls_pos)[1:]
# H = np.concatenate([np.ones((n_loudspeakers, 1), dtype=np.float64),
#                     ls_pos], axis=1)
# g = -np.linalg.norm(ls_pos, ord=2, axis=1)**2 + (c**2)*t_hat**2
# f = (c**2)*(t_hat**2)
# #     print(g)
# #     print(f.shape)
# #     print(f.min())
# print(f.max())
# H_t = np.transpose(H)
# H_inv = np.linalg.inv(H_t @ H)

# sol = np.dot(H_inv @ H_t, g) 


M [2.96524876 2.90915544 2.31004747]


array([22.59220484,  2.96524876,  2.90915544,  2.31004747])